In [ ]:
%%capture

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import calitp_data_analysis.magics
import prep_vp_detour_stops as prep_vp

In [ ]:
# Comment out, this is `parameters` tagged cell
#name = "Montebello Vehicle Positions"

# {name} 

## Potential detour stops 
1. Stop has at least 20% of its scheduled trips served by vehicle positions.
2. There are zero vehicle positions within 10, 25, and 50 meters.
3. There are at least 10 distinct vehicle positions within 100 meters.

In [ ]:
stop_gdf = prep_vp.prep_fct_vp_stop_metrics(
    filters = [[("vp_name", "==", name)]]
)
vp_path = prep_vp.prep_vp_path(
    filters = [[("gtfs_dataset_name", "==", name)]]
) 

intermediate_df = prep_vp.prep_intermediate_vp_stops_trip_crosswalk(
    filters = [[("feed_key", "==", stop_gdf.feed_key.iloc[0])]]
)

In [ ]:
# 10 vp near 100m, this is capturing a lot of rows now
gdf = prep_vp.filter_to_potential_detour_stops(
    stop_gdf,
    intermediate_df,
    vp_path,
    [10, 0, 0, 0, 0.2]
)

print("trip_instance_keys with potential detour stops")
gdf.trip_instance_key.value_counts()

In [ ]:
stop_summary = (
    gdf
    .groupby(["stop_id", "n_vp_near_100m", "pct_vp_trips"])
    .agg({"trip_instance_key": lambda x: list(x)})
    .reset_index()
)

stop_summary

In [ ]:
prep_vp.plot_stops_and_exploded_vp(
    gdf
)

### Robustness and Sensitivity of Cutoffs

More sensitive to what n_vp_cutoff is, rather than pct_vp_trips

In [ ]:
def adjust_filters(n_vp_cutoff, pct_vp):
    test = prep_vp.filter_to_potential_detour_stops(
        stop_gdf,
        intermediate_df,
        vp_path,
        [n_vp_cutoff, 0, 0, 0, pct_vp]
    )

    results = len(test)
    
    return results

In [ ]:
vp_series = []
pct_series = []
n_trips_series = []
for vp in [50, 25, 10]:
    for pct in [0.1, 0.15, 0.2, 0.25, 0.3, 0.35]:
            
        results = adjust_filters(vp, pct)
        vp_series.append(vp)
        pct_series.append(pct)
        n_trips_series.append(results)


results_df = pd.DataFrame()
results_df = results_df.assign(
    at_least_vp = vp_series,
    pct_vp = pct_series,
    n_trips = n_trips_series
)

In [ ]:
results_df